<a href="https://colab.research.google.com/github/Starrysmh/MARK5828-AdvertisingAnalytics-Project/blob/main/4_Image_Processing_Using_Open_CV_signalchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#install openCv
!pip install opencv-contrib-python==4.1.2.30 --user

In [ ]:
#import all required libraries
import cv2 
import glob
import pandas as pd
import numpy as np
import pywt
import re
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# (1) run this code to enable the function to get color attributes

In [ ]:

def brightness(opened_file):
  Bright = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY)

  return round(Bright.mean(), 2)

def saturation(opened_file):
  hsv = cv2.cvtColor(opened_file, cv2.COLOR_BGR2HSV)

  # saturation is the s channel
  s = hsv[:, :, 1]

  return round(s.mean(), 2)

def contrast_of_brightness(opened_file):
  gray = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY)

  return round(gray.std(), 2)


def image_clarity(opened_file):
  gray = cv2.cvtColor(opened_file, cv2.COLOR_BGR2GRAY) 
  bright = gray >= .7

  return round(bright.sum() / bright.size, 2)
  
def warm_hue(opened_file):
  hsv = cv2.cvtColor(opened_file, cv2.COLOR_BGR2HSV)

  # hue is the h channel
  h = hsv[:, :, 0]
  warm = ~ (h > 30) & (h < 210)

  return round(warm.sum() / warm.size, 2)

def image_colorfulness(opened_file):
  # split the image into its respective RGB components
  (B, G, R) = cv2.split(opened_file.astype("float"))

  # compute rg = R - G
  rg = np.absolute(R - G)

  # compute yb = 0.5 * (R + G) - B
  yb = np.absolute(0.5 * (R + G) - B)

  # compute the mean and standard deviation of both `rg` and `yb`
  (rbMean, rbStd) = (np.mean(rg), np.std(rg))
  (ybMean, ybStd) = (np.mean(yb), np.std(yb))

  # combine the mean and standard deviations
  std_root = np.sqrt((rbStd ** 2) + (ybStd ** 2))
  mean_root = np.sqrt((rbMean ** 2) + (ybMean ** 2))

  # derive the "colorfulness" metric and return it
  return round(std_root + (0.3 * mean_root))

(2) composition

In [ ]:
def Composition_variables (image):
    # this function returns the vaules corresponding to
    # 1- Rule of third
    # 2- Diagonal dominance 
    # 3- Vertical physical visual balance
    # 4- Horizontal physical visual balance
    
    
    #__________________________________________________________
    # Rule of third
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliencyMap) = saliency.computeSaliency(image)
    new_arr = ((saliencyMap - saliencyMap.min()) * (1/(saliencyMap.max() - saliencyMap.min()) * 255))
    threshMap = cv2.threshold(new_arr.astype('uint8'), 0, 255, cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(threshMap,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    img_copy = image.copy()
    fx = 0
    fy = 0
    for c in contours:
       # calculate moments for each contour
        M = cv2.moments(c)
        area = cv2.contourArea(c)
        #img_copy = cv2.drawContours(img_copy, contours, contourIdx = -1, 
        #                     color = (255, 0, 0), thickness = 2)
        #print(area)
       # calculate x,y coordinate of center

        if M["m00"]!= 0.0:

            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            fx= fx + cX
            fy = fy + cY

    fx = round(fx/len(contours))
    fy = round(fy/len(contours)) 
    #cv2.circle(image, (fx, fy), 5, (255, 255, 255), -1)
    #cv2.putText(image, "centroid", (fx - 25, fy - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    y,x,c = image.shape
    P = [[y/3, x/3],[y/3, 2*x/3],[2*y/3, x/3],[2*y/3, 2*x/3]]
    dist =[]
    for p in P:
        d = ((fy-p[0])**2+(fx-p[1])**2)**.5
        dist.append(d)
    
    #__________________________________________________________
    # Diagonal dominance 
    xc = fx
    yc = fy

    ycc = y-yc
    yccp = xc*y/x
    theta = np.arctan(y/x)
    d1 = abs((ycc-yccp)*np.cos(theta))
    ycc,yccp,theta,d1

    xcp = x - (x/y*ycc)
    d2 = abs((xc-xcp)*np.sin(theta))
    
    #__________________________________________________________
    #  physical visual balance 
    
    ver_balance = -abs(y/2-fy)
    hor_balance = -abs(x/2-fx)

    return (min(dist), min(d1,d2),-ver_balance,-hor_balance)
  
def horizontal_color_balance(opened_file):
  mid = int(opened_file.shape[1] / 2)
  left_half = np.array(opened_file[:, 0:mid, ], dtype='int')
  right_half = np.flip(np.array(opened_file[:, mid:2 * mid, ],
                                      dtype='int'), axis=1)
  dif_square = np.square(left_half - right_half)
  euclidean = np.sqrt(dif_square.sum(axis=2))

  return round(euclidean.mean(), 2)

def vertical_color_balance(opened_file):
  mid = int(opened_file.shape[0] / 2)
  upper_half = np.array(opened_file[0:mid,: , ], dtype='int')
  lower_half = np.flip(np.array(opened_file[mid:2 * mid,: , ],
                                      dtype='int'), axis=1)
  dif_square = np.square(upper_half - lower_half)
  euclidean = np.sqrt(dif_square.sum(axis=2))

  return round(euclidean.mean(), 2)

(3) figure-background relationshp

In [ ]:
def Difference(opened_file):
    
    #resize image to reduce processing time 
    opened_file = cv2.resize(opened_file, (0, 0), fx=0.5, fy=0.5)
    
    # initialize OpenCV's static fine grained saliency detector and
    # compute the saliency map

    #image = cv2.imread('C:\\Users\\oalq0001\\Downloads\\saliency-detection\\images\\players.jpg')
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliencyMap) = saliency.computeSaliency(opened_file)
    # change the scale into 0-255
    new_arr = ((saliencyMap - saliencyMap.min()) * (1/(saliencyMap.max() - saliencyMap.min()) * 255))

  
    threshMap = cv2.threshold(new_arr.astype('uint8'), 0, 255, cv2.THRESH_OTSU)[1]

    #display objects
    #cv2.imshow("Image", threshMap)
    #cv2.waitKey(0)
    
    kernel1 = np.ones((40,40),np.uint8)
    kernel2 = np.ones((20,20),np.uint8)

    closing = cv2.morphologyEx(threshMap, cv2.MORPH_CLOSE, kernel1)
    threshMap = cv2.erode(closing,kernel2,iterations = 1)

    # calculate Area difference
    objects_size = np.sum(threshMap/255)
    background_size = (threshMap.shape[0]*threshMap.shape[1]) - objects_size
    overall_size = threshMap.shape[0]*threshMap.shape[1]
    
    Area_difference = (objects_size )/(threshMap.shape[0]*threshMap.shape[1])
    
    # calculate Color
    Objects_image = cv2.bitwise_and(opened_file,opened_file,mask = (threshMap/255).astype('uint8'))
    Background_image = cv2.bitwise_and(opened_file,opened_file,mask = ~(threshMap).astype('uint8'))
    
    R_Obj = np.sum(Objects_image[:,:,0])/objects_size
    G_Obj = np.sum(Objects_image[:,:,1])/objects_size
    B_Obj = np.sum(Objects_image[:,:,2])/objects_size
    
    R_back = np.sum(Background_image[:,:,0])/background_size
    G_back = np.sum(Background_image[:,:,1])/background_size
    B_back = np.sum(Background_image[:,:,2])/background_size
    
    color_object = str(R_Obj)+','+str(G_Obj)+','+str(B_Obj)
    color_Background = str(R_back)+','+str(G_back)+','+str(B_back)
    
    # calculate Color difference
    
    Color_difference = ((R_Obj - R_back)**2 + (G_Obj - G_back)**2 + (B_Obj - B_back)**2)**.5
    
    #calculate Texture difference
    edges_objects = cv2.Canny(Objects_image, 50, 150, apertureSize=3)
    edges_background = cv2.Canny(Background_image, 50, 150, apertureSize=3)

    edges_objects_density = np.sum(edges_objects/255)/objects_size
    edges_background_density = np.sum(edges_background/255)/background_size

    Texture_difference = abs(edges_background_density - edges_objects_density)
      
    
    return (Texture_difference,Area_difference, Color_difference)
  
def calc_wavelet (block):
    hsv = cv2.cvtColor(block, cv2.COLOR_BGR2HSV)
    sums =[]
    for i in range(3):
        Channel = hsv[:,:,i]
        coffes_H=pywt.dwt2(Channel,'db1')
        ca,(ch,cv,cd)= coffes_H
        sums.append (np.sum(cd))
    return sums


def Depth_of_field (image):
    
    # Define the window size
    windowsize_r = round(image.shape[0]/4)
    windowsize_c = round(image.shape[1]/4)

    # devide the image into blocks
    window = np.zeros(shape=(windowsize_r,windowsize_c,3,16)).astype('uint8')
    k = 0
    for r in range(0,image.shape[0] - windowsize_r, windowsize_r):
        for c in range(0,image.shape[1] - windowsize_c, windowsize_c):
            window[:,:,:,k] = image[r:r+windowsize_r,c:c+windowsize_c]
            k = k +1
            #display objects

    #center blocks
    center = [5,6,9,10]

    #calaculate the wavelet coeff. for center blocks
    sum_center_blocks = np.zeros(shape = [3])
    for i in center:
        block = window[:,:,:,i]
        Sums = calc_wavelet (block)
        sum_center_blocks = sum_center_blocks + Sums

    #calaculate the wavelet coeff. for whole image 
    Sum_whole_image = calc_wavelet (image)

    #return  the depth of feild for each channel (H,S, and V)
    return abs(sum_center_blocks/Sum_whole_image)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# make sure to put the right path. 

list_of_files_loc = glob.glob('/content/drive/My Drive/Advertising Analytics/Instagram_Images/*.*')
list_of_files = [x for x in list_of_files_loc if re.search('jpg',x)]#if your pictures are not .jpg, then please change ".jpg" to other type

In [ ]:
#check if you get all the pictures
len(list_of_files) #if you have 1000 pictures, then the length of this list of files should be 1000

2265

In [ ]:
#this is the function to extract all 18 features from one picture
def collect_features (filename):
    image = cv2.imread(filename)
    id = filename.split("/")[-1].split('.')[0]
    (Depth_of_field_hue, Depth_of_field_saturation, Depth_of_field_saturation_value) = Depth_of_field (image)
    Brightness = brightness (image)
    Saturation = saturation (image)
    Contrast = contrast_of_brightness (image)
    Clarity = image_clarity (image)
    Warm_hue = warm_hue (image)
    Colorfulness = image_colorfulness (image)
    hor_color = horizontal_color_balance(image)
    ver_color = vertical_color_balance(image)
    (Rule_third, diagonal_dom, physical_ver, physical_hor) = Composition_variables(image)
    (Texture_difference,Area_difference, Color_difference) = Difference(image)
    

    #we extract all the visual features from one picture 
    visual_features  ={'short_code':id,
               'Colorfulness_Picture':Colorfulness,
               'Saturation_Picture': Saturation, 
               'Contrast_Picture':Contrast,
               'Clarity_Picture':Clarity,
               'Brightness_Picture':Brightness,
               'Warm Hue_Picture':Warm_hue,
               
               'ROT_Picture':Rule_third,
               'Diagonal_dominance_Picture':diagonal_dom,
               'Physical_ver_Picture':physical_ver,
               'Physical_Hor_Picture':physical_hor,
               'Color_balance_vertical_Picture':ver_color,
               'Color_balance_horizontal_Picture':hor_color,
               
               
               'Texture_difference_Picture':Texture_difference,
               'Size_difference_Picture':Area_difference,
               'Color_difference_Picture':Color_difference,
               'Depth of field_hue_Picture':Depth_of_field_hue,
               'Depth of field_saturation_Picture':Depth_of_field_saturation,
               'Depth of field_value_Picture':Depth_of_field_saturation_value,
               
              }
    
    
    return visual_features
        

In [ ]:

all_features_combined = []
#loop over list_of_files
for i,filename in enumerate (list_of_files):
    print('processing image no.:',i)
    # send the image name to collect_features function append then results to list
    try:
      all_features = collect_features (filename)
      all_features_combined.append(all_features)
    except Exception as check_error:
      print(check_error)
      continue
     
    
   

processing image no.: 0
processing image no.: 1
'NoneType' object has no attribute 'shape'
processing image no.: 2
processing image no.: 3
processing image no.: 4
processing image no.: 5
'NoneType' object has no attribute 'shape'
processing image no.: 6
processing image no.: 7
processing image no.: 8
processing image no.: 9
'NoneType' object has no attribute 'shape'
processing image no.: 10
processing image no.: 11
processing image no.: 12
processing image no.: 13
processing image no.: 14
processing image no.: 15
'NoneType' object has no attribute 'shape'
processing image no.: 16
processing image no.: 17
processing image no.: 18
processing image no.: 19
processing image no.: 20
processing image no.: 21
processing image no.: 22
'NoneType' object has no attribute 'shape'
processing image no.: 23
processing image no.: 24
processing image no.: 25
'NoneType' object has no attribute 'shape'
processing image no.: 26
processing image no.: 27
processing image no.: 28
processing image no.: 29
pr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in true_divide


processing image no.: 152
processing image no.: 153
processing image no.: 154
'NoneType' object has no attribute 'shape'
processing image no.: 155
processing image no.: 156
processing image no.: 157
processing image no.: 158
processing image no.: 159
processing image no.: 160
processing image no.: 161
processing image no.: 162
processing image no.: 163
processing image no.: 164
processing image no.: 165
processing image no.: 166
'NoneType' object has no attribute 'shape'
processing image no.: 167
processing image no.: 168
processing image no.: 169
processing image no.: 170
processing image no.: 171
processing image no.: 172
processing image no.: 173
processing image no.: 174
processing image no.: 175
processing image no.: 176
processing image no.: 177
processing image no.: 178
processing image no.: 179
processing image no.: 180
processing image no.: 181
processing image no.: 182
processing image no.: 183
processing image no.: 184
processing image no.: 185
processing image no.: 186
proc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


processing image no.: 209
'NoneType' object has no attribute 'shape'
processing image no.: 210
processing image no.: 211
processing image no.: 212
processing image no.: 213
processing image no.: 214
processing image no.: 215
processing image no.: 216
processing image no.: 217
processing image no.: 218
processing image no.: 219
processing image no.: 220
processing image no.: 221
'NoneType' object has no attribute 'shape'
processing image no.: 222
processing image no.: 223
processing image no.: 224
processing image no.: 225
processing image no.: 226
processing image no.: 227
'NoneType' object has no attribute 'shape'
processing image no.: 228
processing image no.: 229
processing image no.: 230
processing image no.: 231
processing image no.: 232
processing image no.: 233
processing image no.: 234
processing image no.: 235
processing image no.: 236
processing image no.: 237
processing image no.: 238
'NoneType' object has no attribute 'shape'
processing image no.: 239
'NoneType' object has 

In [11]:
#check your visual features
all_features_combined[:5]

[{'Brightness_Picture': 201.48,
  'Clarity_Picture': 1.0,
  'Color_balance_horizontal_Picture': 70.13,
  'Color_balance_vertical_Picture': 59.19,
  'Color_difference_Picture': 187.25937814922955,
  'Colorfulness_Picture': 55,
  'Contrast_Picture': 79.12,
  'Depth of field_hue_Picture': 1.2161587526576856,
  'Depth of field_saturation_Picture': 0.06444188722669689,
  'Depth of field_value_Picture': 0.3465045592705169,
  'Diagonal_dominance_Picture': 24.041630560342615,
  'Physical_Hor_Picture': 27.0,
  'Physical_ver_Picture': 61.0,
  'ROT_Picture': 91.82713227702484,
  'Saturation_Picture': 37.7,
  'Size_difference_Picture': 0.451259765625,
  'Texture_difference_Picture': 0.14696259773773862,
  'Warm Hue_Picture': 0.73,
  'short_code': '1598'},
 {'Brightness_Picture': 190.29,
  'Clarity_Picture': 1.0,
  'Color_balance_horizontal_Picture': 92.54,
  'Color_balance_vertical_Picture': 171.23,
  'Color_difference_Picture': 166.85806839746664,
  'Colorfulness_Picture': 33,
  'Contrast_Picture

In [12]:
# save the results in a DataFrame
visual_data = pd.DataFrame(all_features_combined).astype(np.float64)

In [13]:
#check our visual data
visual_data.head()

,short_code,Colorfulness_Picture,Saturation_Picture,Contrast_Picture,Clarity_Picture,Brightness_Picture,Warm Hue_Picture,ROT_Picture,Diagonal_dominance_Picture,Physical_ver_Picture,Physical_Hor_Picture,Color_balance_vertical_Picture,Color_balance_horizontal_Picture,Texture_difference_Picture,Size_difference_Picture,Color_difference_Picture,Depth of field_hue_Picture,Depth of field_saturation_Picture,Depth of field_value_Picture
0,1598.0,55.0,37.70,79.12,1.0,201.48,0.73,91.827132,24.041631,61.0,27.0,59.19,70.13,0.146963,0.451260,187.259378,1.216159,0.064442,0.346505
1,1600.0,33.0,33.04,85.77,1.0,190.29,0.60,104.350371,9.192388,45.0,58.0,171.23,92.54,0.124848,0.377337,166.858068,1.384067,0.304598,0.851598
2,1601.0,41.0,23.81,62.93,1.0,217.82,0.67,113.420555,28.284271,49.0,9.0,106.06,62.52,0.020219,0.791621,51.762340,0.550330,0.103187,2.252280
3,1602.0,60.0,116.26,51.90,1.0,96.86,0.34,92.417531,14.849242,105.0,126.0,128.16,95.57,0.077826,0.643772,43.804585,2.056505,0.429687,0.894040
4,1604.0,76.0,126.69,48.32,1.0,108.10,0.95,93.134312,62.225397,175.0,87.0,104.60,80.90,0.057192,0.237349,107.846292,0.157369,3.343949,6.638462


In [14]:
#then, you can download your visual_data dataframe. use this code to download the the dataframe. 
from google.colab import files

visual_data.to_excel("visual_features_fromOpenCV.xlsx", index=False)
files.download('visual_features_fromOpenCV.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>